# explore_bisNotebook de démonstration pour la brique *encoder* désormais packagée dans `hdc_project.encoder`.Nous importons directement les modules M0–M8 depuis `src/` et rejouons les vérifications clésprésentes dans `explore.ipynb`.

## Initialisation du chemin `src`On s'assure que `src/` est visible dans `sys.path` afin d'importer le package.

In [ ]:
from pathlib import Pathimport sysROOT = Path.cwd().parentSRC = ROOT / "src"if str(SRC) not in sys.path:    sys.path.insert(0, str(SRC))print(f"src path registered: {SRC}")

## Imports des modules packagésLes modules numérotés `M0` à `M7` et le pipeline `M8` sont exposés via `hdc_project.encoder`.

In [ ]:
import numpy as npfrom hdc_project.encoder import m0, m1, m2, m3, m4, m5, m6, m7, pipeline

## Vérification statistique de la clé Rademacher (M0 & M1)On vérifie que deux clés indépendantes ont une similarité proche de 0 et que la probabilité de queues est conforme à la borne de Hoeffding.

In [ ]:
D = 2048n_pairs = 800rng = np.random.default_rng(42)sims = []for _ in range(n_pairs):    J = m0.M0_NewKey(int(rng.integers(0, 2**31 - 1)), D)    Jp = m0.M0_NewKey(int(rng.integers(0, 2**31 - 1)), D)    sims.append(m1.M1_sim(J, Jp))sims = np.asarray(sims)mean_sim = float(sims.mean())tail_prob = float((np.abs(sims) > 0.1).mean())hoeff = 2.0 * np.exp(-D * 0.1**2 / 2.0)print(f"mean(sim) = {mean_sim:.4f}")print(f"P(|sim| > 0.1) = {tail_prob:.4e} (bound={hoeff:.4e})")

## Encodage de phrases avec le pipeline `M8_ENC`On utilise le lexique `M4`, la permutation `M2` et le pipeline `M8` pour encoder quelques phrases,puis on calcule les métriques de validation décrites dans le notebook original.

In [ ]:
D = 4096n = 3rng = np.random.default_rng(123)sentences = [    "hyperdimensional computing is fun",    "vector symbolic architectures are powerful",    "encoding words into hyperspace"]Lex = m4.M4_LexEN_new(seed=1, D=D)pi = rng.permutation(D).astype(np.int64)encoded = pipeline.encode_corpus_ENC(sentences, Lex, pi, D, n, seg_seed0=999)E_list = [entry["E_seq"] for entry in encoded]H_list = [entry["H"] for entry in encoded]print(f"Nombre de segments encodés: {len(encoded)}")print(f"Signature shape: {H_list[0].shape}, dtype: {H_list[0].dtype}")print(f"Accumulateur dtype: {encoded[0]['S'].dtype}")

## Métriques de similarité et courbes de majoritéNous reproduisons les indicateurs `intra/inter` ainsi que les courbes d'erreur de majorité.

In [ ]:
s_intra, s_inter = pipeline.intra_inter_ngram_sims(E_list, D)inter_seg = pipeline.inter_segment_similarity(H_list)print(f"Similarité intra n-gram (moyenne): {s_intra:.4f}")print(f"Similarité inter n-gram |.|: {s_inter:.4f}")print(f"Similarité inter segments |.|: {inter_seg:.4f}")maj_curves = pipeline.majority_error_curve(E_list, pi, D, eta_list=(0.0, 0.05))print("Majority error curve @eta=0.0:", maj_curves[0.0][:3])maj_repeat = pipeline.majority_curve_repeated_vector(E_list, pi, D, eta_list=(0.0, 0.05), trials_per_m=500)print("Repeated-vector curve @eta=0.05:", maj_repeat[0.05][:3])

## Encodage fin : comparaison strict vs unbiased`M8_ENC` permet de choisir entre majorité strict (`strict`) et majority sans biais (`unbiased`).

In [ ]:
tokens = "time flies like an arrow".split()E_uni, X_uni, Xb_uni, S_uni, H_uni = pipeline.M8_ENC(    tokens, pi, n=2, LexEN=Lex, D=D,    majority_mode="unbiased", return_bound=True)E_strict, X_strict, S_strict, H_strict = pipeline.M8_ENC(    tokens, pi, n=2, LexEN=Lex, D=D,    majority_mode="strict", return_bound=False)print(f"#E (unbiased) = {len(E_uni)}, #X = {len(X_uni)}, #Xb = {len(Xb_uni)}")print(f"H_unbiased unique values: {sorted(set(H_uni.tolist()))}")print(f"H_strict unique values: {sorted(set(H_strict.tolist()))}")